In [ ]:
from multiscorer.multiscorer import MultiScorer
#Scikit's libraries for demonstration
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, log_loss, roc_auc_score
from numpy import average
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve,auc
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection  import GridSearchCV
from sklearn.svm import SVC 

import pickle5 as pickle

# GradientBoosting
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, StackingClassifier
from xgboost import XGBClassifier

def confusion_Matrix(classifier, X_test, y_test):
    class_names = ['AMPs', 'NonAMPs']

    disp = plot_confusion_matrix(classifier, X_test, y_test,
                                display_labels = class_names,
                                cmap=plt.cm.Blues, xticks_rotation='vertical')

    disp.ax_.set_title(" Confusion Matrix")

    print(disp.confusion_matrix)
    plt.grid(False)
    plt.show()

In [2]:
#AMPs_df = pd.read_pickle('../../datasets/AMPsNonAMPs_df.239.plk')
AMPs_df = pd.read_pickle('/mnt/vdb/thesis/jax/AMPNonAMP.final.reps')
X= np.array(AMPs_df['reps'].to_list())
y= np.array(AMPs_df['class'].to_list())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Load model 
XGBmodel = XGBClassifier()
XGBmodel.load_model("../Models/best_XGBmodel.model")

with open("../Models/best_RF.model", 'rb') as file:
    RF = pickle.load(file)
    
with open("/mnt/vdb/thesis/best_SVM.rbf.model", 'rb') as file:
    SVM = pickle.load(file)


# Voting

In [3]:
SVM =SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=True)
RF = RandomForestClassifier ( random_state=42)

XGBmodel = XGBClassifier(random_state=42)

In [ ]:
eclf = VotingClassifier(estimators=[
 ('SVM', SVM), ('RF', RF), ('XGBmodel', XGBmodel)],voting='soft',
                        n_jobs =1, weights=[2,1,1],flatten_transform=True)

eclf.fit(X_train, y_train)

[LibSVM]

In [ ]:
y_pred = eclf.predict(X_test)
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
confusion_Matrix(RFclassifier, X_test, y_test)

# Stacking

In [ ]:
estimators = [ ('RF', RF), ('XGBmodel', XGBmodel)]
clf = StackingClassifier(  n_jobs = 24 , 
    estimators=estimators, final_estimator=SVM )
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
confusion_Matrix(RFclassifier, X_test, y_test)